# 时间序列预测

时间序列是随着时间的推移定期收集的数据。时间序列预测是指根据历史数据预测未来数据点的任务。时间序列预测用途很广泛，包括天气预报、零售和销量预测、股市预测，以及行为预测（例如预测一天的车流量）。时间序列数据有很多，识别此类数据中的模式是很活跃的机器学习研究领域。

<img src='notebook_ims/time_series_examples.png' width=80% />

在此 notebook 中，我们将学习寻找时间规律的一种方法，即使用 SageMaker 的监督式学习模型 [DeepAR](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html)。


### DeepAR

DeepAR 使用循环神经网络 (RNN)，它会接受序列数据点作为历史输入，并生成预测序列数据点。这种模型如何学习？

在训练过程中，你需要向 DeepAR estimator 提供训练数据集（由多个时间序列组成）。该 estimator 会查看所有的训练时间序列并尝试发现它们之间的相似性。它通过从训练时间序列中随机抽取**训练样本**进行训练。
* 每个训练样本都由相邻的**上下文**和**预测**窗口（长度已提前固定好）对组成。
    * `context_length` 参数会控制模型能看到过去多久的数据。
    * `prediction_length` 参数会控制模型可以对未来多久做出预测。
    * 详情请参阅[此文档](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_how-it-works.html)。

<img src='notebook_ims/context_prediction_windows.png' width=50% />

> 因为 DeepAR 用多个时间序列进行训练，所以很适合有**循环规律**的数据。

在任何预测任务中，选择的上下文窗口都应该能向模型提供足够的**相关**信息，这样才能生成准确的预测。通常，最接近预测时间帧的数据包含的信息对确定预测结果的影响最大。在很多预测应用中，例如预测月销量，上下文和预测窗口大小一样，但有时候有必要设置更大的上下文窗口，从而发现数据中的更长期规律。

### 能耗数据

在此 notebook 中，我们将使用的数据是全球的家庭耗电量数据。数据集来自 [Kaggle](https://www.kaggle.com/uciml/electric-power-consumption-data-set)，表示从 2006 年到 2010 年的耗电量数据。对于这么庞大的数据集，我们可以预测很长时间的耗电量，例如几天、几周或几个月。预测能耗有很多用途，例如确定耗电量的季节性价格，以及根据预测用量有效地向居民供电。

**趣味阅读**：Google DeepMind 最近展开了一项逆相关项目，他们使用机器学习预测风力发电机产生的电量，并有效地将电力输送给电网。你可以在[这篇帖子](https://deepmind.com/blog/machine-learning-can-boost-value-wind-energy/)中详细了解这项研究。

### 机器学习工作流程

此 notebook 将时间序列预测分成了以下几个步骤：
* 加载和探索数据
* 创建时间序列训练集和测试集
* 将数据变成 JSON 文件并上传到 S3
* 实例化和训练 DeepAR estimator
* 部署模型并创建预测器
* 评估预测器

---

首先加载常规资源。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 加载和探索数据

我们将收集在几年内收集的全球能耗数据。以下单元格将加载并解压缩此数据，并为你提供一个文本数据文件 `household_power_consumption.txt`。

In [ ]:
! wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/March/5c88a3f1_household-electric-power-consumption/household-electric-power-consumption.zip
! unzip household-electric-power-consumption

### 读取 `.txt` 文件

下个单元格显示了文本文件里的前几行，我们可以看看数据格式。

In [ ]:
# display first ten lines of text data
n_lines = 10

with open('household_power_consumption.txt') as file:
    head = [next(file) for line in range(n_lines)]
    
display(head)

## 预处理数据

household_power_consumption.txt 文件具有以下属性：
   * 每个数据点都具有日期和时间记录 (时:分:秒)
   * 各个数据特征用分号 (;) 分隔
   * 某些数据为“nan”或“?”，我们将它们都当做 `NaN` 值

### 处理 `NaN` 值

此 DataFrame 包含一些缺失值的数据点。到目前为止，我们只是丢弃这些值，但是还有其他处理 `NaN` 值的方式。一种技巧是用缺失值所在列的**均值**填充；这样填充的值可能比较符合实际。

我在 `txt_preprocessing.py` 中提供了一些辅助函数，可以帮助你将原始文本文件加载为 DataFrame，并且用各列的平均特征值填充 `NaN` 值。这种技巧对于长期预测来说是可行的，如果是按小时分析和预测，则最好丢弃这些 `NaN` 值或对很小的滑动窗口求平均值，而不是采用整个数据列的平均值。

**在下面的单元格中，我将文件读取为 DataFrame 并用特征级平均值填充 `NaN` 值。**

In [ ]:
import txt_preprocessing as pprocess

# create df from text file
initial_df = pprocess.create_df('household_power_consumption.txt', sep=';')

# fill NaN column values with *average* column value
df = pprocess.fill_nan_with_mean(initial_df)

# print some stats about the data
print('Data shape: ', df.shape)
df.head()

## 全球有效能耗

在此示例中，我们想要预测全球有效能耗，即全球的家庭每分钟平均有效能耗（千瓦）。在下面获取这列数据并显示生成的图形。

In [ ]:
# Select Global active power data
power_df = df['Global_active_power'].copy()
print(power_df.shape)

In [ ]:
# display the data 
plt.figure(figsize=(12,6))
# all data points
power_df.plot(title='Global active power', color='blue') 
plt.show()

因为数据是每分钟记录的，上图包含很多值。所以我只在下面显示了一小部分数据。

In [ ]:
# can plot a slice of hourly data
end_mins = 1440 # 1440 mins = 1 day

plt.figure(figsize=(12,6))
power_df[0:end_mins].plot(title='Global active power, over one day', color='blue') 
plt.show()

### 每小时与每天

每分钟收集了很多数据，我可以通过以下两种方式之一分析数据：
1. 创建很多简短的时间序列，例如一周左右，并且每小时都记录一次能耗，尝试预测接下来的几小时或几天的能耗。
2. 创建更少的很长时间序列，数据每天记录一次，并使用这些数据预测未来几周或几个月的能耗。

两种任务都很有意思。具体取决于你是要预测一天/一周还是更长时间（例如一个月）的规律。鉴于我拥有的数据量，我认为可以查看在多个月或一年内发生的更长重复性趋势。所以我将重采样“全球有效能耗”，将**每日**数据点记录为 24 小时的平均值。

> 我们可以使用 pandas [时间序列工具](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html)根据特定的频率重采样数据，例如按照每小时 ('H') 或每天 ('D') 重采样数据点。



In [ ]:
# resample over day (D)
freq = 'D'
# calculate the mean active power for a day
mean_power_df = power_df.resample(freq).mean()

# display the mean values
plt.figure(figsize=(15,8))
mean_power_df.plot(title='Global active power, mean per day', color='blue') 
plt.tight_layout()
plt.show()

在此图形中，可以看到每年都出现了有趣的趋势。每年初和每年末都会出现能耗高峰，这时候是冬季，供暖和照明使用量都更高。在 8 月份左右也出现小高峰，这时候全球的温度通常更高。

数据依然不够平滑，但是展现了明显的趋势，所以适合用机器学习模型识别这些规律。

---
## 创建时间序列

我的目标是看看能否根据从 2007-2009 的整年数据，准确地预测 2010 年多个月的平均全球有效能耗。

接下来为每个完整的年份数据创建一个时间序列。这只是一种设计决策，我决定使用一整年的数据，从 2007 年 1 月开始，因为 2006 年的数据点不太多，并且这种划分更容易处理闰年。我还可以从第一个收集的数据点开始构建时间序列，只需在下面的函数中更改 `t_start` 和 `t_end` 即可。

函数 `make_time_series` 将为传入的每个年份 `['2007', '2008', '2009']` 创建 pandas `Series`。
* 所有的时间序列将从相同的时间点 `t_start`（或 t0）开始。
    * 在准备数据时，需要为每个时间序列使用一致的起始点；DeepAR 将此时间点作为参考帧，从而学习循环规律，例如工作日的行为与周末不一样，或者夏天与冬天不一样。
    * 你可以更改起始和结束索引，并定义你创建的任何时间序列。
* 在创建时间序列时，我们应该考虑到闰年，例如 2008 年。
* 通常，我们通过从 DataFrame 获取相关的全球能耗数据和日期索引创建 `Series`。

```
# get global consumption data
data = mean_power_df[start_idx:end_idx]

# create time series for the year
index = pd.DatetimeIndex(start=t_start, end=t_end, freq='D')
time_series.append(pd.Series(data=data, index=index))
```

In [ ]:
def make_time_series(mean_power_df, years, freq='D', start_idx=16):
    '''Creates as many time series as there are complete years. This code
       accounts for the leap year, 2008.
      :param mean_power_df: A dataframe of global power consumption, averaged by day.
          This dataframe should also be indexed by a datetime.
      :param years: A list of years to make time series out of, ex. ['2007', '2008'].
      :param freq: The frequency of data recording (D = daily)
      :param start_idx: The starting dataframe index of the first point in the first time series.
          The default, 16, points to '2017-01-01'. 
      :return: A list of pd.Series(), time series data.
      '''
    
    # store time series
    time_series = []
    
    # store leap year in this dataset
    leap = '2008'

    # create time series for each year in years
    for i in range(len(years)):

        year = years[i]
        if(year == leap):
            end_idx = start_idx+366
        else:
            end_idx = start_idx+365

        # create start and end datetimes
        t_start = year + '-01-01' # Jan 1st of each year = t_start
        t_end = year + '-12-31' # Dec 31st = t_end

        # get global consumption data
        data = mean_power_df[start_idx:end_idx]

        # create time series for the year
        index = pd.DatetimeIndex(start=t_start, end=t_end, freq=freq)
        time_series.append(pd.Series(data=data, index=index))
        
        start_idx = end_idx
    
    # return list of time series
    return time_series
    

## 测试结果

下面为每个完整的年份创建一个时间序列，并显示结果。

In [ ]:
# test out the code above

# yearly time series for our three complete years
full_years = ['2007', '2008', '2009']
freq='D' # daily recordings

# make time series
time_series = make_time_series(mean_power_df, full_years, freq=freq)

In [ ]:
# display first time series
time_series_idx = 0

plt.figure(figsize=(12,6))
time_series[time_series_idx].plot()
plt.show()

---
# 按时间拆分数据

我们将用测试数据集评估模型。对于分类等机器学习任务，我们通常随机将样本拆分成不同的数据集，创建训练/测试数据。对于未来预测来说，一定要按照**时间**拆分训练/测试数据，不能按照单个数据点拆分。
> 通常，在创建训练数据时，我们从每个完整的时间序列中去除最后 `prediction_length` 个数据点，并创建训练时间序列。

### 练习：创建训练时间序列

请完成 `create_training_series` 函数，它应该接受完整时间序列数据列表，并返回截断的训练时间序列列表。

* 在此例中，我们想预测一个月的数据，将 `prediction_length` 设为 30 天。
* 为了创建训练数据集，我们将从生成的每个时间序列中去除最后 30 个数据点，所以仅使用第一部分作为训练数据。
* **测试集包含每个时间序列的完整范围**。


In [ ]:
# create truncated, training time series
def create_training_series(complete_time_series, prediction_length):
    '''Given a complete list of time series data, create training time series.
       :param complete_time_series: A list of all complete time series.
       :param prediction_length: The number of points we want to predict.
       :return: A list of training time series.
       '''
    # your code here
        
    pass
    

In [ ]:
# test your code!

# set prediction length
prediction_length = 30 # 30 days ~ a month

time_series_training = create_training_series(time_series, prediction_length)


### 训练和测试序列

我们可以将训练/测试序列绘制到同一个坐标轴上，可视化这些序列。我们应该看到测试序列包含一年的所有数据，训练序列包含最后 `prediction_length` 个数据点之外的数据。

In [ ]:
# display train/test time series
time_series_idx = 0

plt.figure(figsize=(15,8))
# test data is the whole time series
time_series[time_series_idx].plot(label='test', lw=3)
# train data is all but the last prediction pts
time_series_training[time_series_idx].plot(label='train', ls=':', lw=3)

plt.legend()
plt.show()

## 转换为 JSON 

根据 [DeepAR 文档](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html)，DeepAR 要求输入训练数据是 JSON 格式，并包含以下字段：

* **start**：定义时间序列开始日期的字符串，格式为“YYYY-MM-DD HH:MM:SS”。
* **target**：表示时间序列的数值数组。
* **cat**（可选）：类别特征数值数组，可以用于表示记录所属的组。这个字段适合按照项目类别寻找模型，例如对于零售销量，可以将 {'shoes', 'jackets', 'pants'} 表示成类别 {0, 1, 2}。

输入数据的格式应该为，在 JSON 文件中每行一个时间序列。每行看起来像字典，例如：
```
{"start":'2007-01-01 00:00:00', "target": [2.54, 6.3, ...], "cat": [1]}
{"start": "2012-01-30 00:00:00", "target": [1.0, -5.0, ...], "cat": [0]} 
...
```
在上述示例中，每个时间序列都有一个相关的类别特征和一个时间序列特征。

### 练习：格式化能耗数据

对于我们的数据来说：
* 开始日期“start”将为时间序列中第一行的索引，即这一年的 1 月 1 日。
* “target”将为时间序列存储的所有能耗值。
* 我们将不使用可选“cat”字段。

请完成以下实用函数，它应该将 `pandas.Series` 对象转换成 DeepAR 可以使用的相应 JSON 字符串。

In [ ]:
def series_to_json_obj(ts):
    '''Returns a dictionary of values in DeepAR, JSON format.
       :param ts: A single time series.
       :return: A dictionary of values with "start" and "target" keys.
       '''
    # your code here
    
    pass


In [ ]:
# test out the code
ts = time_series[0]

json_obj = series_to_json_obj(ts)

print(json_obj)

### 将数据保存到本地

下面的辅助函数会将一个序列放入 JSON 文件的一行中，并使用换行符“\n”分隔。数据还会编码并写入我们指定的文件名中。

In [ ]:
# import json for formatting data
import json
import os # and os for saving

def write_json_dataset(time_series, filename): 
    with open(filename, 'wb') as f:
        # for each of our times series, there is one JSON line
        for ts in time_series:
            json_line = json.dumps(series_to_json_obj(ts)) + '\n'
            json_line = json_line.encode('utf-8')
            f.write(json_line)
    print(filename + ' saved.')

In [ ]:
# save this data to a local directory
data_dir = 'json_energy_data'

# make data dir, if it does not exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
# directories to save train/test data
train_key = os.path.join(data_dir, 'train.json')
test_key = os.path.join(data_dir, 'test.json')

# write train/test JSON files
write_json_dataset(time_series_training, train_key)        
write_json_dataset(time_series, test_key)

---
## 将数据上传到 S3

接下来，为了使 estimator 能够访问此数据，我将数据上传到 S3。

### Sagemaker 资源

首先指定：
* 训练模型用到的 sagemaker 角色和会话。
* 默认 S3 存储桶，我们可以在其中存储训练、测试和模型数据。

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [ ]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()


### 练习：将训练和测试 JSON 文件上传到 S3

指定唯一的训练和测试 prefix，它们定义了数据在 S3 中的位置。
* 将训练数据上传到 S3 中的某个位置，并将该位置保存到 `train_path`
* 将测试数据上传到 S3 中的某个位置，并将该位置保存到 `test_path`

In [ ]:
# suggested that you set prefixes for directories in S3

# upload data to S3, and save unique locations
train_path = None
test_path = None

In [ ]:
# check locations
print('Training data is stored in: '+ train_path)
print('Test data is stored in: '+ test_path)

---
# 训练 DeepAR Estimator

某些 estimator 具有特定的 SageMaker 构造函数，但是并非都有。你可以创建一个基本 `Estimator` 并传入保存特定模型的特定镜像（或容器）。

接下来，配置要在我们运行模型所在的区域使用的容器镜像。

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

image_name = get_image_uri(boto3.Session().region_name, # get the region
                           'forecasting-deepar') # specify image


### 练习：实例化 Estimator 

现在可以定义将启动训练作业的 estimator 了。一般的 Estimator 将由普通的构造函数参数和 `image_name` 进行定义。
> 你可以查看 [estimator 源代码](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/estimator.py#L595)了解详情。


In [ ]:
from sagemaker.estimator import Estimator

# instantiate a DeepAR estimator
estimator = None


## 设置超参数

接下来，我们需要定义一些 DeepAR 超参数，这些超参数定义了模型大小和训练行为。需要定义周期数评论、预测时长和上下文时长。

* **epochs**：在训练时遍历数据的最大次数。
* **time_freq**：数据集中的时间序列频率（“D”表示每天）。
* **prediction_length**：一个字符串，表示训练模型预测的时间步数量（基于频率单位）。
* **context_length**：模型在做出预测之前可以看到的时间点数量。

### 上下文长度

通常，建议从 `context_length`=`prediction_length` 开始。这是因为 DeepAR 模型还会从目标时间序列那接收“延迟”输入，使模型能够捕获长期依赖关系。例如，每日时间序列可以具有每年季节效应，DeepAR 会自动包含一年延迟。所以上下文长度可以短于一年，模型依然能够捕获这种季节效应。

模型选择的延迟值取决于时间序列的频率。例如，每日频率的延迟值是上一周、上两周、上三周、上四周和一年。详情请参阅 [DeepAR 工作原理文档](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_how-it-works.html)。

### 可选超参数

你还可以配置可选超参数，以进一步优化模型。包括 RNN 模型的层数、每层的单元格数量、似然率函数，以及训练选项，例如批次大小和学习速率。

要了解所有不同 DeepAR 超参数的详尽列表，请参阅 DeepAR [超参数文档](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html)。

In [ ]:
freq='D'
context_length=30 # same as prediction_length

hyperparameters = {
    "epochs": "50",
    "time_freq": freq,
    "prediction_length": str(prediction_length),
    "context_length": str(context_length),
    "num_cells": "50",
    "num_layers": "2",
    "mini_batch_size": "128",
    "learning_rate": "0.001",
    "early_stopping_patience": "10"
}

In [ ]:
# set the hyperparams
estimator.set_hyperparameters(**hyperparameters)

## 训练作业

现在我们可以启动训练作业了。SageMaker 将启动 EC2 实例、从 S3 下载数据、开始训练模型并保存训练过的模型。

如果你提供了 `test` 数据通道（就像在示例中一样），DeepAR 还会计算训练过的模型在此测试数据集上的准确率指标。计算方法是预测测试集中每个时间序列的最后 `prediction_length` 个点，并将它们与时间序列的实际值进行比较。计算的误差指标将包含在日志输出中。

下个单元格可能需要几分钟才能完成，取决于数据大小、模型复杂度和训练选项。

In [ ]:
%%time
# train and test channels
data_channels = {
    "train": train_path,
    "test": test_path
}

# fit the estimator
estimator.fit(inputs=data_channels)

## 部署和创建预测器

训练模型后，我们可以将模型部署到预测器端点上，并使用模型做出预测。

在此 notebook 结束时，记得**删除端点**。我们将在此 notebook 的最后提供一个删除单元格，但是建议提前记住。

In [ ]:
%%time

# create a predictor
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    content_type="application/json" # specify that it will accept/produce JSON
)

---
# 生成预测

根据 DeepAR 的[推理格式](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar-in-formats.html)，`predictor` 要求输入数据是 JSON 格式，并具有以下键：
* **instances**：一个 JSON 格式的时间序列列表，模型应该预测这些时间序列。
* **configuration**（可选）：一个配置信息字典，定义了请求希望的响应类型。

在 configuration 中，可以配置以下键：
* **num_samples**：一个整数，指定了模型在做出概率预测时，生成的样本数。
* **output_types**：一个指定响应类型的列表。我们需要 **quantiles**，它会查看模型生成的 num_samples 列表，并根据这些值为每个时间点生成[分位数估值](https://en.wikipedia.org/wiki/Quantile)。
* **quantiles**：一个列表，指定生成哪些分位数估值并在响应中返回这些估值。


下面是向 DeepAR 模型端点发出 JSON 查询的示例。

```
{
 "instances": [
  { "start": "2009-11-01 00:00:00", "target": [4.0, 10.0, 50.0, 100.0, 113.0] },
  { "start": "1999-01-30", "target": [2.0, 1.0] }
 ],
 "configuration": {
  "num_samples": 50,
  "output_types": ["quantiles"],
  "quantiles": ["0.5", "0.9"]
 }
}
```


## JSON 预测请求

以下代码接受时间序列**列表**作为输入并接受一些配置参数。然后将该序列变成 JSON 实例格式，并将输入转换成相应格式的 JSON_input。

In [ ]:
def json_predictor_input(input_ts, num_samples=50, quantiles=['0.1', '0.5', '0.9']):
    '''Accepts a list of input time series and produces a formatted input.
       :input_ts: An list of input time series.
       :num_samples: Number of samples to calculate metrics with.
       :quantiles: A list of quantiles to return in the predicted output.
       :return: The JSON-formatted input.
       '''
    # request data is made of JSON objects (instances)
    # and an output configuration that details the type of data/quantiles we want
    
    instances = []
    for k in range(len(input_ts)):
        # get JSON objects for input time series
        instances.append(series_to_json_obj(input_ts[k]))

    # specify the output quantiles and samples
    configuration = {"num_samples": num_samples, 
                     "output_types": ["quantiles"], 
                     "quantiles": quantiles}

    request_data = {"instances": instances, 
                    "configuration": configuration}

    json_request = json.dumps(request_data).encode('utf-8')
    
    return json_request

### 获得预测

然后，我们可以使用该函数获得格式化时间序列的预测。

在下个单元格中，我获得了时间序列输入和已知目标，并将格式化输入传入预测器端点中，以获得预测。

In [ ]:
# get all input and target (test) time series
input_ts = time_series_training
target_ts = time_series

# get formatted input time series
json_input_ts = json_predictor_input(input_ts)

# get the prediction from the predictor
json_prediction = predictor.predict(json_input_ts)

print(json_prediction)

## 解码预测

预测器返回 predictor returns JSON 格式的预测，所以我们需要提取可视化结果所需的预测和分位数数据。以下函数会读取 JSON 格式的预测并生成每个分位数中的预测列表。

In [ ]:
# helper function to decode JSON prediction
def decode_prediction(prediction, encoding='utf-8'):
    '''Accepts a JSON prediction and returns a list of prediction data.
    '''
    prediction_data = json.loads(prediction.decode(encoding))
    prediction_list = []
    for k in range(len(prediction_data['predictions'])):
        prediction_list.append(pd.DataFrame(data=prediction_data['predictions'][k]['quantiles']))
    return prediction_list


In [ ]:
# get quantiles/predictions
prediction_list = decode_prediction(json_prediction)

# should get a list of 30 predictions 
# with corresponding quantile values
print(prediction_list[0])

## 显示结果

分位数数据可以提供查看预测结果所需的所有信息。
* 分位数 0.1 和 0.9 表示预测值的上下限。
* 分位数 0.5 表示所有样本预测的中位数。


In [ ]:
# display the prediction median against the actual data
def display_quantiles(prediction_list, target_ts=None):
    # show predictions for all input ts
    for k in range(len(prediction_list)):
        plt.figure(figsize=(12,6))
        # get the target month of data
        if target_ts is not None:
            target = target_ts[k][-prediction_length:]
            plt.plot(range(len(target)), target, label='target')
        # get the quantile values at 10 and 90%
        p10 = prediction_list[k]['0.1']
        p90 = prediction_list[k]['0.9']
        # fill the 80% confidence interval
        plt.fill_between(p10.index, p10, p90, color='y', alpha=0.5, label='80% confidence interval')
        # plot the median prediction line
        prediction_list[k]['0.5'].plot(label='prediction median')
        plt.legend()
        plt.show()

In [ ]:
# display predictions
display_quantiles(prediction_list, target_ts)

## 预测未来

我们没有向模型提供任何 2010 年数据，但是我们看看如果只有已知开始日期，**没有目标**，模型能否预测能耗。

### 练习：为“未来”预测设定请求

请创建一个格式化输入并传入部署的 `predictor`，同时传入常规“configuration”参数。这里的“instances”只有 1 个实例，定义如下：
* **start**：开始时间将为你指定的时间戳。要预测 2010 年的前 30 天，从 1 月 1 日“2010-01-01”开始。
* **target**：目标将为空列表，因为这一年没有完整的相关时间序列。我们特意从模型中去除了该信息，以便测试模型。
```
{"start": start_time, "target": []} # empty target
```

In [ ]:
# Starting my prediction at the beginning of 2010
start_date = '2010-01-01'
timestamp = '00:00:00'

# formatting start_date
start_time = start_date +' '+ timestamp

# format the request_data
# with "instances" and "configuration"
request_data = None


# create JSON input
json_input = json.dumps(request_data).encode('utf-8')

print('Requesting prediction for '+start_time)

然后正常地获取和解码预测响应。

In [ ]:
# get prediction response
json_prediction = predictor.predict(json_input)

prediction_2010 = decode_prediction(json_prediction)


最后，将预测与已知目标序列进行比较。此目标将来自 2010 年的时间序列，我在下面创建了该序列。

In [ ]:
# create 2010 time series
ts_2010 = []
# get global consumption data
# index 1112 is where the 2011 data starts
data_2010 = mean_power_df.values[1112:]

index = pd.DatetimeIndex(start=start_date, periods=len(data_2010), freq='D')
ts_2010.append(pd.Series(data=data_2010, index=index))


In [ ]:
# range of actual data to compare
start_idx=0 # days since Jan 1st 2010
end_idx=start_idx+prediction_length

# get target data
target_2010_ts = [ts_2010[0][start_idx:end_idx]]

# display predictions
display_quantiles(prediction_2010, target_2010_ts)

## 删除端点

请用不同的时间序列尝试你的代码。建议调节 DeepAR 超参数，看看能否改进此预测器的性能。

评估完预测器（任何预测器）后，记得删除端点。

In [ ]:
## TODO: delete the endpoint
predictor.delete_endpoint()

## 总结

你已经见过一个复杂但是应用广泛的时间序列预测方法，并且掌握了将 DeepAR 模型应用到你感兴趣的数据上所需的技能。